In [ ]:
# 환경변수 가져오기
# openai_api_key = os.getenv("OPENAI_API_KEY")
# serpapi_key = os.getenv("SERPAPI_API_KEY")

# 또는 다음과 같이 직접 키 입력 (개발)
# os.environ["OPENAI_API_KEY"] = ""  # 자신의 OpenAI 키
# os.environ["SERPAPI_API_KEY"] = ""

In [ ]:
from dotenv import load_dotenv
import os

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

#### 다음 실습 코드는 학습 목적으로만 사용 바랍니다. 문의 : audit@korea.ac.kr 임성열 Ph.D.

In [ ]:
# 1. 필수 패키지 설치
%pip install langchain-core langchain-openai langchain-community faiss-cpu beautifulsoup4 requests
%pip install --upgrade langchain

In [ ]:
# 2. 환경 설정 및 모듈 임포트
import os
import sqlite3
import requests
from bs4 import BeautifulSoup

from langchain.chat_models import ChatOpenAI
from langchain.vectorstores import FAISS
# from langchain.embeddings import OpenAIEmbeddings은 depreciated
from langchain_openai import OpenAIEmbeddings
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.question_answering import load_qa_chain
from langchain.utilities import SQLDatabase

# OpenAI API Key 설정
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [3]:
# 3. SQLite DB 초기화 함수
# SQLite DB 생성 및 예제 문장 추가
import sqlite3

def initialize_sqlite_db():
    conn = sqlite3.connect("rdb_rag.db")
    cursor = conn.cursor()
    cursor.execute("DROP TABLE IF EXISTS articles;")
    cursor.execute("""
        CREATE TABLE articles (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            title TEXT,
            content TEXT
        );
    """)

    sample_data = [
        (
            "RAG 아키텍처 개요",
            "RAG(Retrieval-Augmented Generation)은 사전 학습된 LLM에 외부 문서 검색 결과를 결합하여 정답을 생성하는 방법론이다. 주로 Dense Retriever + Generator 구성으로 이루어진다."
        ),
        (
            "Vector DB 역할",
            "벡터 데이터베이스는 문서나 쿼리를 임베딩 벡터로 변환하여 유사도 검색을 가능하게 하는 핵심 구성요소이다."
        ),
        (
            "Tool 기반 RAG 확장",
            "ReAct 및 Toolformer 기반 RAG 아키텍처는 외부 툴(SQL, 웹 검색 등)을 능동적으로 호출할 수 있는 Agent를 포함한다."
        )
    ]

    cursor.executemany("INSERT INTO articles (title, content) VALUES (?, ?)", sample_data)
    conn.commit()
    conn.close()


In [4]:
# 4. 문서 임베딩 함수
def embed_document(file_path="attention.pdf", db_path="faiss_index"):
    loader = PyPDFLoader(file_path)
    documents = loader.load()

    splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
    chunks = splitter.split_documents(documents)

    embeddings = OpenAIEmbeddings()
    vector_db = FAISS.from_documents(chunks, embeddings)
    vector_db.save_local(db_path)

    return f"Document embedded and saved to {db_path}"

In [5]:
# 5. 문서 검색 함수
from langchain.vectorstores import FAISS

def retrieve_documents(query, db_path="faiss_index", k=3):
    embeddings = OpenAIEmbeddings()
    db = FAISS.load_local(
        db_path, 
        embeddings,
        allow_dangerous_deserialization=True)  # <-- 역직렬화 허용 추가
    retriever = db.as_retriever(search_kwargs={"k": k})
    return retriever.invoke(query)


In [6]:
# 6. 검색된 문서 요약 함수
%pip install pypdf

from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableSequence
from langchain_core.documents import Document

def summarize_chunks(chunks):
    llm = ChatOpenAI(temperature=0)
    
    prompt = PromptTemplate.from_template("""
    다음 문서를 읽고 간단히 요약해 하라:\n\n{context}
    """)
    
    # 문서 내용을 하나로 합침
    merged = "\n\n".join([doc.page_content for doc in chunks])
    
    chain: RunnableSequence = (
        RunnablePassthrough.assign(context=lambda x: merged)
        | prompt
        | llm
    )
    
    return chain.invoke({})


Note: you may need to restart the kernel to use updated packages.


In [7]:
# 7. SQL 검색 함수
def sql_query_tool(query):
    db = SQLDatabase.from_uri("sqlite:///rdb_rag.db")
    return db.run(f"""
        SELECT content FROM articles
        WHERE title LIKE '%{query}%' OR content LIKE '%{query}%'
    """)

    return results


In [8]:
# 8. 웹 스크래핑 함수
%pip install google-search-results
from serpapi import GoogleSearch

# 환경변수 설정 예 (노트북 상에서 직접), https://serpapi.com 에서 개인 계정을 생성하고 본인 API를 받아서 입력

os.environ["SERPAPI_API_KEY"] = "9c61eeb3bac82aea465b08d257b1a419dbf02d1734808b80cae1e881ae796196"

def web_scraper(query, num_results=5):
  try:
    serpapi_key = os.getenv("SERPAPI_API_KEY")  # 환경변수 또는 직접 입력
    params = {
        "engine": "google",
        "q": query,
        "hl": "ko",
        "gl": "kr",
        "api_key": serpapi_key
    }

    search = GoogleSearch(params)
    results = search.get_dict()
    
# 결과 유효성 체크
    if "organic_results" not in results or not results["organic_results"]:
        return "검색 결과 없음"

# 상위 결과 제목 요약 추출
    titles = [res.get("title", "제목 없음") for res in results["organic_results"][:num_results]]
    return "\n".join(f"- {title}" for title in titles)
  except Exception as e:
   return f"웹 검색 오류: {str(e)}"


  Using cached google_search_results-2.4.2-py3-none-any.whl
Note: you may need to restart the kernel to use updated packages.


In [10]:
# 9. 전체 RAG 통합 파이프라인
def rag_pipeline(user_query):
    # 1. Vector DB 검색
    retrieved = retrieve_documents(user_query)
    summary = summarize_chunks(retrieved).content
    source_vector = "[VectorDB 요약]\n" + summary

    # 2. 웹 검색
    scraped = web_scraper(user_query)
    source_web = "[웹 검색 결과]\n" + scraped

    # 3. SQL 검색
    sql_result = sql_query_tool(user_query)
    source_sql = "[SQL 검색 결과]\n" + sql_result

    # 4. 전체 정보를 종합한 프롬프트 생성
    full_prompt = f"""
    사용자 질문: {user_query}

    다음은 다양한 소스로부터 수집된 정보이다. 이를 바탕으로 정확하고 간결한 답변을 생성하라.

    {source_vector}

    {source_web}

    {source_sql}

    참고한 출처는 답변 마지막에 모두 제시하라. 답변은 한국어 경어체로 사용하라.
    """

    llm = ChatOpenAI(temperature=0.3)
    final_response = llm.invoke(full_prompt)

    # 5. 출처와 함께 결과 반환
    return {
        "answer": final_response,
        "sources": {
            "vector_summary": summary,
            "sql_result": sql_result,
            "web_result": scraped
        }
    }

In [11]:
# 10. 실행: 전체 초기화 및 질문 테스트
# Step 1: DB 생성 및 문서 임베딩 (최초 1회만)
initialize_sqlite_db()
embed_document("attention.pdf")  # 반드시 현재 위치에 attention.pdf 파일이 있어야 함

# Step 2: 질문 실행
query = "RAG 아키텍처"
answer = rag_pipeline(query)
print("최종 응답:\n", answer['answer'].content)


/var/folders/xr/0xbgl9wd3wz914nd58ylmykw0000gp/T/ipykernel_97210/2368931604.py:9: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(temperature=0)


최종 응답:
 RAG 아키텍처는 Retrieval-Augmented Generation의 약자로, 사전 학습된 LLM에 외부 문서 검색 결과를 결합하여 정답을 생성하는 방법론을 말합니다. 주로 Dense Retriever + Generator 구성으로 이루어져 있습니다. 이 아키텍처는 ReAct 및 Toolformer를 기반으로 하며, 외부 툴(SQL, 웹 검색 등)을 능동적으로 호출할 수 있는 Agent를 포함하고 있습니다.

[출처: VectorDB, 웹 검색 결과, SQL 검색 결과]


In [12]:
# 출처별 상세 결과 확인
from IPython.display import display, Markdown

print("출처별 정보\n" + "-"*40)

# Vector 요약
print("\n [Vector 요약]")
print("-" * 40)
print(answer["sources"].get("vector_summary", "(결과 없음)"))

# SQL 결과
print("\n [SQL 결과]")
print("-" * 40)
print(answer["sources"].get("sql_result", "(결과 없음)"))

# 웹 검색 결과
print("\n [웹 검색 결과]")
print("-" * 40)
print(answer["sources"].get("web_result", "(결과 없음)"))

출처별 정보
----------------------------------------

 [Vector 요약]
----------------------------------------
다양한 논문들이 다양한 주제에 대해 연구를 진행하고 있으며, 주로 인공지능 및 기계학습 분야에 관련된 주제들을 다루고 있다. 예를 들어, LSTM 네트워크에 대한 요인화 기법, 어텐션 네트워크, Adam 최적화 방법, 그리고 신경망을 사용한 기계 번역 등이 연구되고 있다. 이러한 논문들은 주로 국제 학회나 학술지에 발표되었으며, 최신 기술 및 방법론에 대한 연구를 다루고 있다.

 [SQL 결과]
----------------------------------------
[('RAG(Retrieval-Augmented Generation)은 사전 학습된 LLM에 외부 문서 검색 결과를 결합하여 정답을 생성하는 방법론이다. 주로 Dense Retriever + Generator 구성으로 이루어진다.',), ('ReAct 및 Toolformer 기반 RAG 아키텍처는 외부 툴(SQL, 웹 검색 등)을 능동적으로 호출할 수 있는 Agent를 포함한다.',)]

 [웹 검색 결과]
----------------------------------------
- 검색 증강 생성(Retrieval Augmented Generation, RAG)이란?
- RAG 아키텍처의 이해
- Retrieval-Augmented Generation(RAG)의 흐름과 아키텍쳐
- 기업용 RAG 아키텍처 - 네이버 블로그
- GPT는 모르면 지어낸다? RAG는 진짜 정보를 찾아온다


In [13]:
print(web_scraper("RAG 아키텍처란 무엇인가?"))


- RAG란? - 검색 증강 생성 AI 설명
- 검색 증강 생성(RAG)이란? 생성형 AI의 정확도를 높이는 기술
- 검색 증강 생성(RAG)이란 무엇인가?
- RAG 아키텍처의 이해
- GPT는 모르면 지어낸다? RAG는 진짜 정보를 찾아온다


In [14]:
import os
print(os.getenv("SERPAPI_API_KEY"))

9c61eeb3bac82aea465b08d257b1a419dbf02d1734808b80cae1e881ae796196


In [15]:
# https://serpapi.com 검색 API 동작 테스트 (모든 상세한 검색, 구조화 응답 제공)
# 참고로 https://serper.dev 는 다른 사이트입니다. (경량화 요약 검색 제공)
from serpapi import GoogleSearch

params = {
    "engine": "google",
    "q": "RAG 아키텍처란 무엇인가",
    "api_key": "9c61eeb3bac82aea465b08d257b1a419dbf02d1734808b80cae1e881ae796196"
}
search = GoogleSearch(params)
print(search.get_dict())  # 전체 응답 확인


{'search_metadata': {'id': '68b15dc3f5d5ac22b1bf3ef1', 'status': 'Success', 'json_endpoint': 'https://serpapi.com/searches/71f18118467ac231/68b15dc3f5d5ac22b1bf3ef1.json', 'pixel_position_endpoint': 'https://serpapi.com/searches/71f18118467ac231/68b15dc3f5d5ac22b1bf3ef1.json_with_pixel_position', 'created_at': '2025-08-29 07:58:59 UTC', 'processed_at': '2025-08-29 07:58:59 UTC', 'google_url': 'https://www.google.com/search?q=RAG+%EC%95%84%ED%82%A4%ED%85%8D%EC%B2%98%EB%9E%80+%EB%AC%B4%EC%97%87%EC%9D%B8%EA%B0%80&oq=RAG+%EC%95%84%ED%82%A4%ED%85%8D%EC%B2%98%EB%9E%80+%EB%AC%B4%EC%97%87%EC%9D%B8%EA%B0%80&sourceid=chrome&ie=UTF-8', 'raw_html_file': 'https://serpapi.com/searches/71f18118467ac231/68b15dc3f5d5ac22b1bf3ef1.html', 'total_time_taken': 12.19}, 'search_parameters': {'engine': 'google', 'q': 'RAG 아키텍처란 무엇인가', 'google_domain': 'google.com', 'device': 'desktop'}, 'search_information': {'query_displayed': 'RAG 아키텍처란 무엇인가', 'total_results': 6020, 'time_taken_displayed': 0.22, 'organic_res